In [1]:
# bob.refresh {"cron": "10 9,12,16,18,20,22 * * *", "timeout": 400000}

In [2]:
import bob
hc = bob.HealthCheck()
hc_key = "bfc41c24-d278-4bc8-8d95-767e5da44f59"
start = hc.start(hc_key)
print(start)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/bfc41c24-d278-4bc8-8d95-767e5da44f59, 2020-04-07


In [3]:
import json
from datetime import datetime, timedelta, date
import time
import pandas as pd

#-- INIT : Declare local variable
#rep
input_folder = "../data_input/"
output_folder = "../data_output/"
param_folder = "../../_parameters/"

#-> Date
now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

#-- STEP0 : Connect to mongodb
#Get credentials info
credentials = param_folder + 'mongodb.json'
with open(credentials) as f:
    dbCon = json.load(f)

mongo_host = dbCon["host"]
mongo_port = dbCon["port"]
mongo_username = dbCon["username"]
mongo_password = dbCon["password"]

#Connect to mongo
MongoDB = bob.mongo.connect(mongo_host,mongo_port,mongo_username,mongo_password)
db = 'app-wsr-v1'

Successfully connected to MongoDB


In [4]:
#-- Transform yahoo finance data DB_ALL (VALUE, VAR, VARP)
db_all = bob.mongo.read_df('DB_ALL','yahoo').drop('_id', axis=1)
def get_all(df):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'SCENARIO': pd.to_datetime(df['DATE'], format='%Y-%m-%d').dt.strftime('%d/%m/%Y'),
        'DATE_ORDER': domain['DATE_ORDER'],
        'LABEL': domain['LABEL'],
        'GROUPS': domain['GROUPS'],
        'TYPE': domain['TYPE'],
        'VALUE': domain['CLOSE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'VOLUME': domain['VOLUME'],
        'UNIT_VALUE': domain['UNIT'],
        'UNIT_VARP': ' %',
        'LAST_UPDATE': now
    })
    return domain.reset_index(drop=True)

In [7]:
#-- Economy Highlighted KPIS 
domain021 = get_all(db_all)
# bob.mongo.save_df(domain021,'021',db,True)
domain021 = domain021[(domain021['SCENARIO'].isin(['07/04/2020','06/04/2020','03/04/2020','02/04/2020','01/04/2020']))]
domain021.to_csv(output_folder + '021.csv', sep=';')
domain021.head(5)

,ENTITY,SCENARIO,DATE_ORDER,LABEL,GROUPS,TYPE,VALUE,VAR,VARP,VOLUME,UNIT_VALUE,UNIT_VARP,LAST_UPDATE
0,WORLDWIDE,07/04/2020,20200407,NIKKEI,NIKKEI,INDICES,18950.179688,373.878907,2.012666,0.0,pts,%,07/04/2020 10:38:52
1,WORLDWIDE,06/04/2020,20200406,NIKKEI,NIKKEI,INDICES,18576.300781,756.111328,4.243004,105700.0,pts,%,07/04/2020 10:38:52
2,WORLDWIDE,03/04/2020,20200403,NIKKEI,NIKKEI,INDICES,17820.189453,1.468750,0.008243,96000.0,pts,%,07/04/2020 10:38:52
3,WORLDWIDE,02/04/2020,20200402,NIKKEI,NIKKEI,INDICES,17818.720703,-246.689453,-1.365535,107300.0,pts,%,07/04/2020 10:38:52
4,WORLDWIDE,01/04/2020,20200401,NIKKEI,NIKKEI,INDICES,18065.410156,-851.599610,-4.501767,105800.0,pts,%,07/04/2020 10:38:52


In [5]:
#-- Transform yahoo finance data DB_TREND
db_trend = bob.mongo.read_df('DB_TREND','yahoo')
def get_trend(df, variable):
    domain = df.copy()[df['TYPE'] == variable]
    domain = pd.DataFrame({
        'ENTITY': 'WORLDWIDE',
        'DATE': domain['DATE'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'GROUPS': domain['LABEL'],
        'BOTTOM_FILTER': domain['DATE_SCENARIO'],
        'UPPER_FILTER_M': domain['METRIC'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
        'OPEN': domain['OPEN'],
        'HIGH': domain['HIGH'],
        'LOW': domain['LOW'],
        'CLOSE': domain['CLOSE'],
        'VOLUME': domain['VOLUME'],
        'META1': domain['META1'],
        'META2': domain['META2'],
        'LAST_UPDATE': now
    })
    return domain.reset_index(drop=True)

In [6]:
#-- Economy - Indice
domain201 = get_trend(db_trend, 'INDICES')
domain201.to_csv(output_folder + '201.csv', sep=';')
bob.mongo.save_df(domain201,'201',db,True)

domain201.head(5)

Dataframe 201 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.5625770092010498 secnds ---


,ENTITY,DATE,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,OPEN,HIGH,LOW,CLOSE,VOLUME,META1,META2,LAST_UPDATE
0,WORLDWIDE,2020-04-03,20200403,NIKKEI,Last 14 days,DAILY VARIATION (VALUE),1.468750,pts,"+,.2f",17951.439453,18059.150391,17646.5,17820.189453,0.0,1h00 à 7h00,NaN,03/04/2020 17:57:10
1,WORLDWIDE,2020-04-03,20200403,NIKKEI,Last 30 days,DAILY VARIATION (VALUE),1.468750,pts,"+,.2f",17951.439453,18059.150391,17646.5,17820.189453,0.0,1h00 à 7h00,NaN,03/04/2020 17:57:10
2,WORLDWIDE,2020-04-03,20200403,NIKKEI,Last 360 days,DAILY VARIATION (VALUE),1.468750,pts,"+,.2f",17951.439453,18059.150391,17646.5,17820.189453,0.0,1h00 à 7h00,NaN,03/04/2020 17:57:10
3,WORLDWIDE,2020-04-03,20200403,NIKKEI,Last 14 days,DAILY VARIATION (%),0.008243,%,"+,.2f",17951.439453,18059.150391,17646.5,17820.189453,0.0,1h00 à 7h00,NaN,03/04/2020 17:57:10
4,WORLDWIDE,2020-04-03,20200403,NIKKEI,Last 30 days,DAILY VARIATION (%),0.008243,%,"+,.2f",17951.439453,18059.150391,17646.5,17820.189453,0.0,1h00 à 7h00,NaN,03/04/2020 17:57:10


In [7]:
#-- Economy - Currencies
domain202 = get_trend(db_trend, 'CURRENCIES')
domain202.to_csv(output_folder + '202.csv', sep=';')
bob.mongo.save_df(domain202,'202',db,True)

upper_filter_r202 = domain202[['GROUPS']].drop_duplicates().sort_values(by="GROUPS",ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_r202,'202_UPPER_FILTER_R',db,True)

domain202.head(5)

Dataframe 202 successfully save in database app-wsr-v1 in MongoDB. Time: --- 9.893583059310913 secnds ---
Dataframe 202_UPPER_FILTER_R successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.012903928756713867 secnds ---


,ENTITY,DATE,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,OPEN,HIGH,LOW,CLOSE,VOLUME,META1,META2,LAST_UPDATE
0,WORLDWIDE,2020-04-03,20200403,USD/SGD,Last 14 days,DAILY VARIATION (VALUE),0.004160,,"+,.2f",1.43024,1.439,1.4289,1.43856,0.0,NaN,NaN,03/04/2020 17:57:10
1,WORLDWIDE,2020-04-03,20200403,USD/SGD,Last 30 days,DAILY VARIATION (VALUE),0.004160,,"+,.2f",1.43024,1.439,1.4289,1.43856,0.0,NaN,NaN,03/04/2020 17:57:10
2,WORLDWIDE,2020-04-03,20200403,USD/SGD,Last 360 days,DAILY VARIATION (VALUE),0.004160,,"+,.2f",1.43024,1.439,1.4289,1.43856,0.0,NaN,NaN,03/04/2020 17:57:10
3,WORLDWIDE,2020-04-03,20200403,USD/SGD,Last 14 days,DAILY VARIATION (%),0.290017,%,"+,.2f",1.43024,1.439,1.4289,1.43856,0.0,NaN,NaN,03/04/2020 17:57:10
4,WORLDWIDE,2020-04-03,20200403,USD/SGD,Last 30 days,DAILY VARIATION (%),0.290017,%,"+,.2f",1.43024,1.439,1.4289,1.43856,0.0,NaN,NaN,03/04/2020 17:57:10


In [8]:
#-- Economy - Equities
domain203 = get_trend(db_trend, 'EQUITIES')
domain203.to_csv(output_folder + '203.csv', sep=';')
bob.mongo.save_df(domain203,'203',db,True)

upper_filter_r203 = domain203[['GROUPS']].drop_duplicates().sort_values(by="GROUPS",ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_r203,'203_UPPER_FILTER_R',db,True)

domain203.head(5)

Dataframe 203 successfully save in database app-wsr-v1 in MongoDB. Time: --- 23.689698457717896 secnds ---
Dataframe 203_UPPER_FILTER_R successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.013803720474243164 secnds ---


,ENTITY,DATE,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,OPEN,HIGH,LOW,CLOSE,VOLUME,META1,META2,LAST_UPDATE
0,WORLDWIDE,2020-04-03,20200403,Exxon Mobil Corporation,Last 14 days,DAILY VARIATION (VALUE),0.590000,$,"+,.2f",41.450001,42.169998,40.889999,40.990002,9355999.0,Energy,Nasdaq,03/04/2020 17:57:10
1,WORLDWIDE,2020-04-03,20200403,Exxon Mobil Corporation,Last 30 days,DAILY VARIATION (VALUE),0.590000,$,"+,.2f",41.450001,42.169998,40.889999,40.990002,9355999.0,Energy,Nasdaq,03/04/2020 17:57:10
2,WORLDWIDE,2020-04-03,20200403,Exxon Mobil Corporation,Last 360 days,DAILY VARIATION (VALUE),0.590000,$,"+,.2f",41.450001,42.169998,40.889999,40.990002,9355999.0,Energy,Nasdaq,03/04/2020 17:57:10
3,WORLDWIDE,2020-04-03,20200403,Exxon Mobil Corporation,Last 14 days,DAILY VARIATION (%),1.460396,%,"+,.2f",41.450001,42.169998,40.889999,40.990002,9355999.0,Energy,Nasdaq,03/04/2020 17:57:10
4,WORLDWIDE,2020-04-03,20200403,Exxon Mobil Corporation,Last 30 days,DAILY VARIATION (%),1.460396,%,"+,.2f",41.450001,42.169998,40.889999,40.990002,9355999.0,Energy,Nasdaq,03/04/2020 17:57:10


In [9]:
#-- Economy - Commodities
#Data source
domain204 = get_trend(db_trend, 'COMMODITIES')
domain204.to_csv(output_folder + '204.csv', sep=';')
bob.mongo.save_df(domain204,'204',db,True)

upper_filter_r204 = domain204[['GROUPS']].drop_duplicates().sort_values(by="GROUPS",ascending=True).reset_index(drop=True)
bob.mongo.save_df(upper_filter_r204,'204_UPPER_FILTER_R',db,True)

domain204.head(5)

Dataframe 204 successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.5511600971221924 secnds ---
Dataframe 204_UPPER_FILTER_R successfully save in database app-wsr-v1 in MongoDB. Time: --- 0.016774892807006836 secnds ---


,ENTITY,DATE,DATE_ORDER,GROUPS,BOTTOM_FILTER,UPPER_FILTER_M,VALUE,UNIT_VALUE,PRECISION,OPEN,HIGH,LOW,CLOSE,VOLUME,META1,META2,LAST_UPDATE
0,WORLDWIDE,2020-04-03,20200403,Silver,Last 14 days,DAILY VARIATION (VALUE),0.010000,$,"+,.2f",14.635,14.71,14.475,14.625,3475684.0,NaN,NaN,03/04/2020 17:57:10
1,WORLDWIDE,2020-04-03,20200403,Silver,Last 30 days,DAILY VARIATION (VALUE),0.010000,$,"+,.2f",14.635,14.71,14.475,14.625,3475684.0,NaN,NaN,03/04/2020 17:57:10
2,WORLDWIDE,2020-04-03,20200403,Silver,Last 360 days,DAILY VARIATION (VALUE),0.010000,$,"+,.2f",14.635,14.71,14.475,14.625,3475684.0,NaN,NaN,03/04/2020 17:57:10
3,WORLDWIDE,2020-04-03,20200403,Silver,Last 14 days,DAILY VARIATION (%),0.068423,%,"+,.2f",14.635,14.71,14.475,14.625,3475684.0,NaN,NaN,03/04/2020 17:57:10
4,WORLDWIDE,2020-04-03,20200403,Silver,Last 30 days,DAILY VARIATION (%),0.068423,%,"+,.2f",14.635,14.71,14.475,14.625,3475684.0,NaN,NaN,03/04/2020 17:57:10


In [2]:
done = hc.done(hc_key)
print(done)

End ==> send to https://health.cashstory.com/ping/bfc41c24-d278-4bc8-8d95-767e5da44f59, 2020-04-06
